In [81]:
#!pip install z3-solver

In [82]:
from itertools import combinations
from utils import import_instances, plot_solution, output_solution
from z3 import *

## Data Import

In [83]:
# Get all instances from text files
instances = import_instances('input/instances/')

['ins-1.txt', 'ins-2.txt', 'ins-3.txt', 'ins-4.txt', 'ins-5.txt', 'ins-6.txt', 'ins-7.txt', 'ins-8.txt', 'ins-9.txt', 'ins-10.txt', 'ins-11.txt', 'ins-12.txt', 'ins-13.txt', 'ins-14.txt', 'ins-15.txt', 'ins-16.txt', 'ins-17.txt', 'ins-18.txt', 'ins-19.txt', 'ins-20.txt', 'ins-21.txt', 'ins-22.txt', 'ins-23.txt', 'ins-24.txt', 'ins-25.txt', 'ins-26.txt', 'ins-27.txt', 'ins-28.txt', 'ins-29.txt', 'ins-30.txt', 'ins-31.txt', 'ins-32.txt', 'ins-33.txt', 'ins-34.txt', 'ins-35.txt', 'ins-36.txt', 'ins-37.txt', 'ins-38.txt', 'ins-39.txt', 'ins-40.txt']


## SAT

In [84]:
# Functions
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one(solver, bool_vars):
    solver.add(at_most_one(bool_vars))
    solver.add(at_least_one(bool_vars))

def get_variables(instance_number):
    # Get the number of blocks
    number_of_blocks = int(instances[instance_number][1])
    
    # Get block lengths and heights
    blocks_width = []
    blocks_height = []

    for value in instances[instance_number][2:]:
        width, height = value.split(' ')
        blocks_width.append(int(width))
        blocks_height.append(int(height))  

    width = int(instances[instance_number][0])
    return number_of_blocks, blocks_width, blocks_height, width


In [85]:
def vlsi(s, height):
    # Variables
    p = [[[Bool(f"x_{i}_{j}_{n}") for n in range((2* number_of_blocks) + 1)] for j in range(height)] for i in range(width)]

    # A cell has only one value
    for i in range(width):
        for j in range(height):
            exactly_one(s, p[i][j])

    for n in range(number_of_blocks):
        # A circuit has only one position
        exactly_one(s, [p[i][j][n] for i in range(width) for j in range(height)])

        # A circuit stays inside the plane
        s.add(at_least_one([p[i][j][n] for i in range(width - blocks_width[n] + 1) for j in range(height - blocks_height[n] + 1)]))

    # The circuits can't overlap
    for n in range(number_of_blocks):
        for i in range(width - blocks_width[n]+1):
            for j in range(height - blocks_height[n]+1):
                for k in range(i, i + blocks_width[n]):
                    for u in range(j, j + blocks_height[n]):
                        if(k != i or u != j):
                            s.add(Implies(p[i][j][n], p[k][u][n + number_of_blocks]))

    sol = []
    if s.check() == sat:
        m = s.model()
        for i in range(width):
            sol.append([])
            for j in range(height):
                for k in range(2*(number_of_blocks) + 1):
                    if m.evaluate(p[i][j][k]):
                        sol[i].append(k)
        print(sol)
    elif s.reason_unknown() == "timeout":
        print("Solver timeout")
    else:
        print("Failed to solve")
    return sol

In [ ]:
for instance_number in range(40):
    print(instance_number)
    number_of_blocks, blocks_width, blocks_height, width = get_variables(instance_number)
    starting_height = int(sum([blocks_width[i] * blocks_height[i] for i in range(number_of_blocks)]) / width)
    height_i = starting_height
    s = Solver()

    times = 300 * 1000 # 300 sec
    s.set(timeout=times)

    sol = vlsi(s, height_i)
        
    if (sol) :
        start_x = []
        start_y = []
        for n in range(number_of_blocks):
            for i in range(len(sol)):
                for j in range(len(sol[0])):
                    if sol[i][j] == n:
                        start_x.append(i)
                        start_y.append(j)
        circuits = [[blocks_width[i], blocks_height[i], start_x[i], start_y[i]] for i in range(number_of_blocks)]
        plot_solution(width, height_i, circuits, f'output/SAt/images/out-{instance_number+1}.png')
        output_solution(instances[instance_number], height_i, start_x, start_y, f'output/SAT/solutions/out-{instance_number + 1}.txt')